In [80]:
import numpy as np
import cv2

In [97]:
def resize_bilinear_1(image, scale):
    # empty list to store each resized rgb channel image(total 3)
    image_rgb = []

    B = np.array([[0, 1], [1, 1]])
    B_inv = np.linalg.inv(B)

    # perform bilinear interpolation for each color channel(RGB)
    for i in range(3):
        image_channel = image[:, :, i]

        # size of original image
        n_row_in, n_column_in = image_channel.shape

        # size of output image after scaling
        n_row_out = n_row_in * scale
        n_column_out = n_column_in * scale

        # allocate space for output image
        image_output = np.zeros((n_row_out, n_column_out), dtype=np.uint8)

        # pad original image to make sure every pixel in the original image can find
        # 4 nearby data points for bilinear interpolation
        bottom_row = image_channel[-1, :]
        image_row_padding = np.vstack((image_channel, bottom_row))

        rightmost_column = image_row_padding[:, -1]
        image_padding = np.c_[image_row_padding, rightmost_column]

        # preallocate space for F matrix
        # F stores f matrix for each pixel in the original image matrix (computed using image after padding)
        # each f is a 2 by 2 matrix that stores 4 pixel values for bilinear interpoolation
        F = np.zeros((n_row_in, n_column_in, 2, 2))

        for x in range(n_row_in):
            for y in range(n_column_in):
                f = np.array([[image_padding[x][y],   image_padding[x][y+1]],
                              [image_padding[x+1][y], image_padding[x+1][y+1]]])
                F[x][y] = f

        # interpolate image pixel by pixel
        for x in range(n_row_out):
            # the corresponding x position in the original image matrix
            x_ori = (x / n_row_out) * n_row_in

            # x_ori's integer part (Used to index F matrix created above)
            x_int = int(np.floor(x_ori))

            # the amount of value that need to be interpolated
            x_interp = x_ori - np.floor(x_ori)

            for y in range(n_column_out):
                # the corresponding y position in the original image matrix
                y_ori = (y / n_column_out) * n_column_in

                # y_ori's integer part (Used to index F matrix created above)
                y_int = int(np.floor(y_ori))

                # the amount of value that need to be interpolated
                y_interp = y_ori - np.floor(y_ori)

                if x_interp==0.0 and y_interp==0.0:
                    image_output[x][y] = image_channel[int(x_ori)][int(y_ori)]
                else:
                    # interpolate value in x direction (row vector)
                    X = np.expand_dims(np.array([x_interp**1, x_interp**0]), axis=0)
                    # interpolate value in y direction (column vector)
                    Y = np.expand_dims(np.array([y_interp**1, y_interp**0]), axis=1)

                    F_interp = F[x_int][y_int]

                    value = X.dot(B_inv).dot(F_interp).dot(B_inv).dot(Y)

                    # after bilinear interpolation floating point values are returned intead of
                    # unsigned integer value between 0 and 255. Thus we need to clamp these value
                    # between 0 and 255 in order to show it as an image
                    if value < 0:
                        value = 0
                    elif value > 255:
                        value = 255

                    image_output[x][y] = value
        
        image_rgb.append(image_output)

    image_rgb_output = cv2.merge((image_rgb[0], image_rgb[1], image_rgb[2]))

    return image_rgb_output

In [82]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(a)
bottom_row = a[-1, :]
b = np.vstack((a, bottom_row))
print(b)
rightmost_column = b[:, -1]
c = np.c_[b, rightmost_column]
print(c)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 2 3]
 [4 5 6]
 [7 8 9]
 [7 8 9]]
[[1 2 3 3]
 [4 5 6 6]
 [7 8 9 9]
 [7 8 9 9]]


In [83]:
print(np.floor(0.5))
print(1.5 - np.floor(1.5))

0.0
0.5


In [84]:
a = [1,2,3]
b = [4,5,6]
c = []
c.append(b)
c.append(a)
print(c[0])

[4, 5, 6]


In [85]:
image = cv2.imread("../Test_Image/man.png")

In [100]:
image_resized = resize_bilinear_1(image, 16)

In [101]:
cv2.imshow("Resized (Bilinear)", image_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1